<a href="https://colab.research.google.com/github/pepezamarripa/datascience_certificate/blob/main/RetoEmpleados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos las librerías necesarias

In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA



In [ ]:
# Importo el fichero empleadosReto.csv
archivo_csv = "empleadosRETO.csv"

# Creo el DF EmpleadosAttrtion
EmpleadosAttrition = pd.read_csv(archivo_csv)

# Extraigo las características del DF
caracteristicas_EmpleadosAttrition = EmpleadosAttrition.dtypes

# Muestro los las características
print(caracteristicas_EmpleadosAttrition)

# Muestro los primeros valores del DF
print(EmpleadosAttrition.head())

Eliminar las columnas que no tienen relación con la salida deseada

In [ ]:
# Elimino las columnas
columnas_eliminar = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']
EmpleadosAttrtion = EmpleadosAttrition.drop(columns=columnas_eliminar)

# Muestro las columnas
print(caracteristicas_EmpleadosAttrition)

# Muestro los primeros valores del DF
print(EmpleadosAttrition.head())

Creo la columna YearsInCompany para determinar cuánto tiempo tiene un trabajador en la compañía.

La columna HiringDate tiene fechas en formato MM/DD/YY y MM/DD/YYYY. Para porder calcular la columna YearsInCompany es necesario procesar las fechas que tienen el formato MM/DD/YY

In [ ]:
# Creo la columna HiringYear con los últimos dos dígitos de HiringDate
EmpleadosAttrition['HiringYear'] = EmpleadosAttrition['HiringDate'].str[-2:]

# Obtengo el año actual en formato 'YY'
currentYear = datetime.now().strftime('%y')

# Comparo el valor de HiringYear con currentYear.
# Si el valor de currentYear > a HiringYear => agrego un '20' antes de HiringYear.
# En caso contrario, agrego un '19'
EmpleadosAttrition['HiringYear'] = EmpleadosAttrition.apply(lambda row: '20' + row['HiringYear'] if row['HiringYear'] < currentYear else '19' + row['HiringYear'], axis=1)

# Ahora, calcularé la oolumna YearsAtCompany
# Obtengo el año actual en formato 'YYYY'
currentYear = datetime.now().strftime('%Y')

# Creo la columna YearsAtCompany calculando la diferencia de años
EmpleadosAttrition['YearsAtCompany'] = pd.to_numeric(currentYear) - pd.to_numeric(EmpleadosAttrition['HiringYear'])


Calcular la columna DistanceFromHome

In [ ]:
# Renombro la columna DistanceFromHome a DistanceFromHome_km
EmpleadosAttrition.rename(columns={'DistanceFromHome': 'DistanceFromHome_km'}, inplace=True)

# Creo la nueva columna DistanceFromHome eliminando el término 'km' y convirtiéndola en integer
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.extract('(\d+)').astype(int)


In [ ]:
# Elimino las columnas HiringDate, HiringYear y DistanceFromHome_km
columnas_a_eliminar = ['HiringDate', 'HiringYear', 'DistanceFromHome_km']
EmpleadosAttrition = EmpleadosAttrition.drop(columnas_a_eliminar, axis=1)


Creación del DataFrame SueldoPromedioDepto

In [ ]:
# Creo el DataFrame SueldoPromedioDepto con el promedio de MonthlyIncome por Department
SueldoPromedioDepto = EmpleadosAttrition.groupby('Department')['MonthlyIncome'].mean().reset_index()
SueldoPromedioDepto.rename(columns={'MonthlyIncome' : 'SueldoPromedio'}, inplace=True)

print(SueldoPromedioDepto)

Escalar la columna MonthlyIncome

In [ ]:
scaler = MinMaxScaler()
EmpleadosAttrition['MonthlyIncome_scaled'] = scaler.fit_transform(EmpleadosAttrition[['MonthlyIncome']])

Convertir las variables categóricas a variables numéricas

In [ ]:
# Aplico Label Encoding a todas las variables categóricas
label_encoder = LabelEncoder()

EmpleadosAttrition['BusinessTravel_Label'] = label_encoder.fit_transform(EmpleadosAttrition['BusinessTravel'])
EmpleadosAttrition['Department_Label'] = label_encoder.fit_transform(EmpleadosAttrition['Department'])
EmpleadosAttrition['EducationField_Label'] = label_encoder.fit_transform(EmpleadosAttrition['EducationField'])
EmpleadosAttrition['Gender_Label'] = label_encoder.fit_transform(EmpleadosAttrition['Gender'])
EmpleadosAttrition['JobRole_Label'] = label_encoder.fit_transform(EmpleadosAttrition['JobRole'])
EmpleadosAttrition['MaritalStatus_Label'] = label_encoder.fit_transform(EmpleadosAttrition['MaritalStatus'])
EmpleadosAttrition['OverTime_Label'] = label_encoder.fit_transform(EmpleadosAttrition['OverTime'])
EmpleadosAttrition['Attrition_Label'] = label_encoder.fit_transform(EmpleadosAttrition['Attrition'])


# Elimino las columnas originales categóricas
columnas_categoricas = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime', 'Attrition']
EmpleadosAttrition = EmpleadosAttrition.drop(columnas_categoricas, axis=1)


Calcular la correlación lineal de cada una de las variables con respecto a Attrition.

In [ ]:
# Calculo la correlación de Pearson para cada variable con respecto a Attrition_Label
correlaciones = EmpleadosAttrition.corr()['Attrition_Label'].sort_values(ascending=False)

# Muestro las correlaciones
print(correlaciones)

In [ ]:
# Selecciono las variables con correlación mayor o igual a 0.1 en valores absolutos
variables_seleccionadas = correlaciones[abs(correlaciones) >= 0.1].index
print(variables_seleccionadas)

# Creo el DataFrame EmpleadosAttritionFinal con las variables seleccionadas
EmpleadosAttritionFinal = EmpleadosAttrition[variables_seleccionadas]

In [ ]:
# Aplico el PCA a las variables seleccionadas
pca = PCA()
EmpleadosAttritionPCA_array = pca.fit_transform(EmpleadosAttritionFinal)

# Determino el número mínimo de componentes para explicar el 80% de la varianza
varianza_acumulativa = pca.explained_variance_ratio_.cumsum()
num_componentes = (varianza_acumulativa < 0.8).sum() + 1

# Obtengo solo el número necesario de componentes principales
EmpleadosAttritionPCA_min = pd.DataFrame(EmpleadosAttritionPCA_array[:, :num_componentes], columns=[f'C{i}' for i in range(num_componentes)])

# Ajusto el orden de las columnas en EmpleadosAttritionFinal
columnas_ordenadas = list(EmpleadosAttritionFinal.columns) + list(EmpleadosAttritionPCA_min.columns)
EmpleadosAttritionFinal = EmpleadosAttritionFinal[columnas_ordenadas]

# Guardo el DataFrame en el fichero csv EmpleadosAttritionFinal
EmpleadosAttritionFinal.to_csv("EmpleadosAttritionFinal.csv", index=False)